<a href="https://colab.research.google.com/github/hikkaaaa/machine-learning/blob/main/handling_missing_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1st task:
# Clean up the data
I got the idea from a 5 day missing values clean up challenge from kaggle

In [5]:
import numpy as np
import pandas as pd

#the dataset was taken from kaggle
#the data is about San Francisko permits
df = pd.read_csv("./drive/MyDrive/Colab Notebooks/Building_Permits.csv")

#for reproductibility:
np.random.seed(0)

/tmp/ipython-input-5-2520874443.py:6: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./drive/MyDrive/Colab Notebooks/Building_Permits.csv")


In [6]:
#take a look at some of the data:
df.sample(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
40553,201403039652,8,otc alterations permit,03/03/2014,3732,008,400,NaN,Clementina,St,...,NaN,NaN,1.0,constr type 1,NaN,6.0,South of Market,94103.0,"(37.780460571778164, -122.40450626524974)",1334094491645
169731,201510159735,3,additions alterations or repairs,10/15/2015,2609,028,79,NaN,Buena Vista,Tr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94117.0,"(37.76757916496494, -122.43793170417105)",1399356139170
19180,M409787,8,otc alterations permit,07/22/2013,4624,031,178,NaN,West Point,Rd,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73524725436046, -122.38063828309745)",1311685491725
68047,201411191888,8,otc alterations permit,11/19/2014,0039,109,294,NaN,Francisco,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.805257822817126, -122.40998545760392)",1362881288870
64238,M527228,8,otc alterations permit,10/14/2014,1251,002,707,NaN,Cole,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Haight Ashbury,94117.0,"(37.76836885973765, -122.45074431487859)",135886493776


In [20]:
#how many missing values do we have in each column
missing_values_count = df.isnull().sum()
missing_values_count

#by analyzing the data, if we take 2 columns as examples, namely say: street number suffix and zipcode
#not all streets can have their number suffixes, so probably nan values in this field are simply bcus they dont exist
#however sipcode should be present for all streets, therfore the nan values in this column were probably forgotten to be recorded

,0
Permit Number,0
Permit Type,0
Permit Type Definition,0
Permit Creation Date,0
Block,0
Lot,0
Street Number,0
Street Number Suffix,196684
Street Name,0
Street Suffix,2768


In [23]:
#if you are in a hurry, drop the rows or columsn that contain missing values (however we will not do that, since analyzing the table is better)

#but just for education reasons, in case if you wanna drop the rows:
#df.dropna()
#in case if you want to drop all the columns that contain nan
#df.dropna(axis = 1)

In [27]:
#now lets try to get a subset that we gonna be working with and try to fill in some data to nan values
subset = df.loc[:, 'Street Number Suffix':'Zipcode'].head()
subset

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode
0,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0
2,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN,...,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,2.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0


In [31]:
#replacing all the NA's with the value that comes after it in the same column
subset.fillna(method = "bfill", inplace = True)
subset

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode
0,NaN,Ellis,St,0.0,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,2.0,NaN,3.0,constr type 3,1.0,constr type 1,NaN,3.0,Tenderloin,94102.0
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,2.0,NaN,3.0,constr type 3,1.0,constr type 1,NaN,3.0,Tenderloin,94102.0
2,NaN,Pacific,Av,0.0,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017,...,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,2.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0


In [32]:
#now lets fill the remaining NA's (the ones that didnt have any values to replace themselves with) with 0
subset.fillna(0, inplace = True)
subset

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode
0,0.0,Ellis,St,0.0,0.0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,2.0,0.0,3.0,constr type 3,1.0,constr type 1,0.0,3.0,Tenderloin,94102.0
1,0.0,Geary,St,0.0,0.0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,2.0,0.0,3.0,constr type 3,1.0,constr type 1,0.0,3.0,Tenderloin,94102.0
2,0.0,Pacific,Av,0.0,0.0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017,...,2.0,0.0,1.0,constr type 1,1.0,constr type 1,0.0,3.0,Russian Hill,94109.0
3,0.0,Pacific,Av,0.0,0.0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,2.0,0.0,5.0,wood frame (5),5.0,wood frame (5),0.0,3.0,Nob Hill,94109.0
4,0.0,Market,St,0.0,0.0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,2.0,0.0,3.0,constr type 3,0.0,0,0.0,6.0,Tenderloin,94102.0
